API: https://api.openweathermap.org/data/2.5/weather?q=dublin,353&appid=f800ca2d2726566b082da6aad95fe093

In [34]:
import datetime
import traceback
import os
import requests
import time
import json
import pyodbc
import textwrap
import pytz

Setting up pyodbc

In [35]:
# driver
driver = '{ODBC Driver 17 for SQL Server}'

In [36]:
# server and db name
server_name = 'dublinbus-team12-server'
database_name = 'dublinbus-team12-db'

In [37]:
# server
server = '{}.database.windows.net,1433'.format(server_name)

In [38]:
# username and passord 
username = 'innovationgeeks'
password = 'Laurawillsaveus445!'

In [39]:
# connection string 
connection_string = textwrap.dedent('''
    Driver={driver};
    Server={server};
    Database={database};
    Uid={username};
    Pwd={password};
    Encrypt=yes;
    TrustServerCertificate=no;
    Connection Timeout=30;
'''.format(
    driver = driver, 
    server = server, 
    database = database_name,
    username = username,
    password = password
    )                               
)

In [40]:
# pyodbc conncetion object
conn: pyodbc.Connection = pyodbc.connect(connection_string)

In [41]:
# curser object for connection 
cursor: pyodbc.Cursor = conn.cursor()

Functions

In [42]:
owm_current_weather = '''
    CREATE TABLE weather_current(
        lon FLOAT,
        lat FLOAT,
        weather_id INT,
        weather_main VARCHAR(256),
        weather_description VARCHAR(256),
        weather_icon VARCHAR(256),
        main_temp FLOAT,
        main_feels_like FLOAT,
        main_temp_min FLOAT,
        main_temp_max FLOAT,
        main_pressure INT,
        main_humidity INT,
        visability INT,
        wind_speed FLOAT,
        wind_deg INT,
        wind_gust FLOAT,
        clouds_all INT,
        dt INT,
        sys_type INT,
        sys_id INT,
        sys_sunrise INT,
        sys_sunset INT
    )'''

In [43]:
def create_table():
    '''function to create table in azure db - only needs to be created once'''
    try:
        cursor.execute(owm_current_weather)
        cursor.commit()
    except Exception as e:
        print(e)

In [44]:
# create_table() # Only needs to be created once

In [45]:
def get_data(data):
    ''' Function to return the data from the json object'''
    
    now = datetime.datetime.utcnow()
    
    return (
        data.get("coord").get("lon"),
        data.get("coord").get("lat"),
        data.get("weather")[0].get("id"),
        data.get("weather")[0].get("main"),
        data.get("weather")[0].get("description"),
        data.get("weather")[0].get("icon"),
        data.get("main").get("temp"),
        data.get("main").get("feels_like"),
        data.get("main").get("temp_min"),
        data.get("main").get("temp_max"),
        data.get("main").get("pressure"),
        data.get("main").get("humidity"),
        data.get("visibility"),
        data.get("wind").get("speed"),
        data.get("wind").get("deg"),
        data.get("wind").get("gust"),
        data.get("clouds").get("all"),
        data.get("dt"),
        data.get("sys").get("type"),
        data.get("sys").get("id"),
        data.get("sys").get("sunrise"),
        data.get("sys").get("sunset")
    )

In [46]:
def current_weather():
    ''' function to get data form owm and import it into the database'''
    r = requests.get("https://api.openweathermap.org/data/2.5/weather?q=dublin,353&appid=f800ca2d2726566b082da6aad95fe093")
    data = json.loads(r.text)
    print("this is data", data)
    vals = get_data(data)
    print("this is vals", vals)
    cursor.execute('''INSERT INTO dbo.weather_current (lon,lat,weather_id,weather_main,weather_description,weather_icon,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,visability,wind_speed,wind_deg,wind_gust,clouds_all,dt,sys_type,sys_id,sys_sunrise,sys_sunset) VALUES {}'''.format(vals))
    cursor.commit()
    return

In [ ]:
def main():
    ''' Main function'''
    while True: 
        try:
            current_weather()
            time.sleep(60 * 60)
        except:
            print(traceback.format_exc())
            print("Error found an error when querying")
            time.sleep(60 * 60)
            
if __name__ == "__main__":
    main()

In [33]:
cursor.execute("DROP TABLE dbo.weather_current")
conn.commit()